### Sentiment Analysis of Instagram posts 
Script copywrite: Aman, Jayedi;
Written: 20210212

#### take a look here#
https://colab.research.google.com/drive/1xyaAMav_gTo_KvpHrO05zWFhmUaILfEd?usp=sharing#scrollTo=UKZAgDk9Jnrx

In [ ]:
!nvidia-smi

In [ ]:
#Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install the package
!pip install transformers

In [ ]:
#Import sentiment model from hugging face
from transformers import pipeline
import torch.nn.functional as F
import torch
import torchvision

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
classifier = pipeline('sentiment-analysis', model = model_name, device = 0) #connecting with GPU using device variable




In [ ]:
import sys
from os import listdir
from os.path import isfile, join
import pandas as pd
import os
import time
import csv

path_in= r'/content/drive/MyDrive/MIZZOU/CAADRIA2022/Paper2/SentimentAnalysis/63861 posts from 116 locations processed/'

all_files = [f for f in listdir(path_in)]

filenames = list(filter(lambda x: x[-5:] == ('.xlsx') or x[-4:] == ('.xls'), all_files))
filenames.sort()
#print(jpg_files) 
print("1.Total Excel Files in the folder:",len(filenames))

1.Total Excel Files in the folder: 111


In [ ]:
filenames[110:111]

In [ ]:
##**Read the data for sentiment analysis
#Read the data
FileID = 110 #change to 0
for files in filenames[110:111]: #comment this line and remove comment from the next line
#for files in filenames:
  startTime = time.time() # initiate timer
  data = pd.read_excel(path_in+filenames[FileID])
  
  #Read the raw data
  CaptionDataRaw = data.values.T[3].astype(str).tolist()
  #Clean the data and keep the first 1500 characters for keeping the tensors 
  CaptionData = [item[:1200] for item in CaptionDataRaw]
  CaptionData


  ##**Sentiment analysis 
  SentimentResults = []

  #Sentiment analysis using defined model
  for list in CaptionData:
    #results = RunModel([list])
    results = classifier([list])
    for result in results:
      print(result)
      SentimentResults.append(result)
  print(SentimentResults)

  #making list of each variables of the dictionary
  label= []
  score= []
  for i in SentimentResults[0:len(SentimentResults)]:
      label.append(str(i['label']))
      score.append(str(i['score']))
  #print(label,score)

  ##**Rank count for the spreadsheet after the sentiment analysis
  RankCount_5 = label.count('5 stars')
  print(RankCount_5)
  RankCount_4 = label.count('4 stars')
  print(RankCount_4)
  RankCount_3 = label.count('3 stars')
  print(RankCount_3)
  RankCount_2 = label.count('2 stars')
  print(RankCount_2)
  RankCount_1 = label.count('1 star')
  print(RankCount_1)

    #Read other data for csv writeup
  Date = data.values.T[1].astype(str).tolist()
  Time = data.values.T[2].astype(str).tolist()
  User_ID = data.values.T[11].astype(str).tolist()
  LocationName = data.values.T[12].astype(str).tolist()
  Latitude = data.values.T[14].astype(str).tolist()
  Longitude = data.values.T[15].astype(str).tolist()


    #Convert the list into panda dataframe and write a backup csv file
  workspace = '/content/drive/MyDrive/MIZZOU/CAADRIA2022/Paper2/SentimentAnalysis/SentimentOutputResults'
  csv_sentiments = os.path.join(workspace,"%s_Sentiment results_Ranking.csv"%filenames[FileID])
  list_dict = {'Date': Date,'Time': Time,'User_ID': User_ID,'LocationName': LocationName,'Longitude': Longitude,'Latitude': Latitude,'Caption': CaptionData, 'Rank': label, 'Score': score}
  df = pd.DataFrame(list_dict)
  df
  try: 
      df.to_csv(csv_sentiments, index=False,encoding='utf-8')
      print('Data is written in the folder')
  except IOError:
      print('File is not written in the folder')

  # csv_SentimentCounts = os.path.join(workspace,"0_count_Sentiment results_Ranking.csv")
  # list_dict2 = {'FileName':filenames[FileID], 'Longitude': Longitude,'Latitude': Latitude,'Rank_5': RankCount_5, 'Rank_4': RankCount_4, 'Rank_3': RankCount_3, 'Rank_2': RankCount_2, 'Rank_1': RankCount_1}
  # df2 = pd.DataFrame(list_dict2)
  # df2
  # df2.to_csv(csv_SentimentCounts, index=False,encoding='utf-8')
  fields = [filenames[FileID], LocationName[0], Latitude[0], Longitude[0], RankCount_5, RankCount_4, RankCount_3, RankCount_2, RankCount_1]
  with open(r'/content/drive/MyDrive/MIZZOU/CAADRIA2022/Paper2/SentimentAnalysis/SentimentOutputResults/0.csv','a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)

  FileID +=1


  endTime = time.time()
  #print('Processing', filename, 'ID:', fileID,"runtime:",endTime-startTime)
print("All Done")
